## Google Photos API - Download images from Google Photos using Python

Using the Google Photos REST API you can download, upload and modify images stored in Google Photos.

The following steps describe how to set up a simple project that lets you use Python to download images from Google Photos:

In [5]:
%%capture capt 
#saves the output to variable capt, to print output capt.stdout, capt.stderr
!pip install -r "requirements.txt"
!pip freeze > requirements.txt

In [2]:
!which python
!which pip

/Users/andyflury/opt/anaconda3/bin/python
/Users/andyflury/opt/anaconda3/bin/pip


## Use the Google Photo Library API for the first time:

The following section shows how to use OAuth Credentials for authentication with the Google Library API. The code section below covers the following steps:

8. Create a service for the first time:

    1. Initialize GooglePhotosApi `google_photos_api = GooglePhotosApi()`

    2. Create Service using the `client_secret.json` file: `service = google_photos_api.create_service()`
        
        
       <b>Calling the API for the first time:</b>
       1. Google will ask you if you want to grant the App the required permissions you defined with the scope:
       ![](read_me_img/sign_in_google_acc.png)
       2. Since its just a test app at the moment, Google will make you aware of that > Click on "Continue"
       3. Once you granted the app the required permissions, you will see a "token_......pickle" file created in the folder "credentials". This token file will be used for future calls.

In [1]:
import pickle
import os
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
import requests

class GooglePhotosApi:
    def __init__(self,
                 api_name = 'photoslibrary',
                 client_secret_file= r'./credentials/client_secret.json',
                 api_version = 'v1',
                 scopes = ['https://www.googleapis.com/auth/photoslibrary']):
        '''
        Args:
            client_secret_file: string, location where the requested credentials are saved
            api_version: string, the version of the service
            api_name: string, name of the api e.g."docs","photoslibrary",...
            api_version: version of the api

        Return:
            service:
        '''

        self.api_name = api_name
        self.client_secret_file = client_secret_file
        self.api_version = api_version
        self.scopes = scopes
        self.cred_pickle_file = f'./credentials/token_{self.api_name}_{self.api_version}.pickle'

        self.cred = None

    def run_local_server(self):
        # is checking if there is already a pickle file with relevant credentials
        if os.path.exists(self.cred_pickle_file):
            with open(self.cred_pickle_file, 'rb') as token:
                self.cred = pickle.load(token)

        # if there is no pickle file with stored credentials, create one using google_auth_oauthlib.flow
        if not self.cred or not self.cred.valid:
            if self.cred and self.cred.expired and self.cred.refresh_token:
                self.cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(self.client_secret_file, self.scopes)
                self.cred = flow.run_local_server()

            with open(self.cred_pickle_file, 'wb') as token:
                pickle.dump(self.cred, token)
        
        return self.cred


## Initialize photos api and create service

In [2]:
GOOGLE_PHOTOS_API = GooglePhotosApi()
CREDS = GOOGLE_PHOTOS_API.run_local_server()
PHOTOS_HEADERS = {
    'content-type': 'application/json',
    'Authorization': 'Bearer {}'.format(CREDS.token)
}

def init_gp_server():
    global GOOGLE_PHOTOS_API, CREDS, PHOTO_HEADERS
    GOOGLE_PHOTOS_API = GooglePhotosApi()
    CREDS = GOOGLE_PHOTOS_API.run_local_server()
    PHOTOS_HEADERS = {
        'content-type': 'application/json',
        'Authorization': 'Bearer {}'.format(CREDS.token)
    }
init_gp_server()

## Use pythons requests module and the token file to retrieve data from Google Photos

The functions here
1. request media items from Hilledwight shared album and store the item IDs in picIds.txt
2. request media items from all my photos organized by month and store the items IDs respective txt files in the idFiles directory

We're limiting the number of pictures requested by numPics in each function, but in the future we'll remove numPics

In [3]:
import json
import requests

GET_LIBRARIES_URL = 'https://photoslibrary.googleapis.com/v1/albums'
MEDIA_ITEMS_URL = 'https://photoslibrary.googleapis.com/v1/mediaItems:search'

MAX_NUM_PICS = 5200000000
MONTHS = [
    {'month': 8, 'year':2021, 'name':'August'}, 
    {'month': 9, 'year':2021, 'name': 'September'}, 
    {'month': 10, 'year':2021, 'name': 'October'}, 
    {'month': 11, 'year':2021, 'name': 'November'}, 
    {'month': 12, 'year':2021, 'name': 'December'}, 
    {'month': 1, 'year':2022, 'name': 'January'}, 
    {'month': 2, 'year':2022, 'name': 'February'}, 
    {'month': 3, 'year':2022, 'name': 'March'},
    {'month': 4, 'year':2022, 'name': 'April'}, 
    {'month': 5, 'year':2022, 'name': 'May'}, 
    {'month': 6, 'year':2022, 'name': 'June'}, 
]

CATEGORIES = [
    ['ANIMALS','PETS','ANIMALS'],
    ['FASHION'],
    ['LANDMARKS'],
    ['ARTS'],
    ['FLOWERS','GARDENS','LANDSCAPES', 'NATURE'],
    ['BIRTHDAYS'],
    ['FOOD'],
    ['NIGHT'],
    ['SELFIES'],
    ['CITYSCAPES', 'HOUSES', 'CITYSCAPES'],
    ['PEOPLE'],
    ['SPORT'],
    ['HOLIDAYS'],
    ['CRAFTS'],
    ['PERFORMANCES'],
    ['TRAVEL'],
    ['RECEIPTS','WEDDINGS','WHITEBOARDS','SCREENSHOTS','UTILITY','DOCUMENTS', 'MISC']
]

def writeToFile(f, mediaItems):
    for item in mediaItems:
        try:
            f.write('%s\n' %item['id'])
        except:
            print('!!!! WARNING: Write error !!!!')

def get_hd_people_pics():
    print('Downloading HD pics...')
    try:
        res = requests.request("GET", GET_LIBRARIES_URL, headers=PHOTOS_HEADERS)
        albumID = res.json()['albums'][1]['id']
    except:
        print('!!!! WARNING: Library request error !!!!') 
        print(res)
        return
    
    payload = {
      "albumId": albumID,
      "pageSize": "25"
    }
    res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
    res = res.json()
    if 'error' in res:
        print('!!!! WARNING: Library request error !!!!')
        print(res)
        return
    numPics = 0
    with open('idFiles/picIDs.txt', 'w+') as f:
        while 'nextPageToken' in res and numPics < MAX_NUM_PICS:
            if 'mediaItems' in res:
                writeToFile(f, res['mediaItems'])
            numPics += len(res['mediaItems'])
            payload = {
              "albumId": albumID,
              "pageSize": "25",
              "pageToken": res['nextPageToken'],
            }
            res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
            res = res.json()
            if 'error' in res:
                print('!!!! WARNING: Library request error !!!!')
                print(res)
                return
    f.close()
    print('Download complete!')
    return numPics

def get_month_pics(monthEntry):
    payload = {
      "filters": {
        "dateFilter": {
          "dates": [
            {
              "month": monthEntry['month'],
              "year": monthEntry['year']
            }
           ]
            }
          },
        "pageSize": "25"
    }
    
    res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
    res = res.json()
    if 'error' in res:
        print('!!!! WARNING: Library request error !!!!')
        print(res)
        return
    numPics = 0
        
    with open('idFiles/months/'+monthEntry['name']+'PicIDs.txt', 'w+') as f:
        while 'nextPageToken' in res and numPics < MAX_NUM_PICS:
            if 'mediaItems' in res:
                writeToFile(f, res['mediaItems'])
            payload = {
              "filters": {
                "dateFilter": {
                  "dates": [
                    {
                      "month": monthEntry['month'],
                      "year": monthEntry['year']
                    }
                   ]
                }
              },
              "pageSize": "25",
              "pageToken": res['nextPageToken']
            }
            res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
            res = res.json()
            if 'error' in res:
                print('!!!! WARNING: Library request error !!!!')
                print(res)
                return
            if 'mediaItems' in res:
                numPics += len(res['mediaItems'])
            
    f.close()
    return numPics

def get_all_month_pics():
    print('Downloading pics with monthly filter...')
    for monthEntry in MONTHS:
        get_month_pics(monthEntry)
    print('Download complete!')
        
        
def get_category_pics(category):
    categoryLabel = category[0]
    if len(category)>1:
        categoryLabel = category[-1]
        category = category[0:len(category)-1]
    payload = {
      "filters": {
        "contentFilter": {
          "includedContentCategories": category
        }
      },
      "pageSize": "25"
    }
    
    res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
    res = res.json()
    if 'error' in res:
        print('!!!! WARNING: Library request error !!!!')
        print(res)
        return
    numPics = 0
    with open('idFiles/categories/'+categoryLabel+'PicIDs.txt', 'w+') as f:
        while 'nextPageToken' in res and numPics < MAX_NUM_PICS:
            if 'mediaItems' in res:
                writeToFile(f, res['mediaItems'])
            payload = {
              "filters": {
                "contentFilter": {
                  "includedContentCategories": category
                }
              },
              "pageSize": "25",
              "pageToken": res['nextPageToken']
            }
            
            res = requests.request("POST", MEDIA_ITEMS_URL,  data=json.dumps(payload), headers=PHOTOS_HEADERS)
            res = res.json()
            if 'error' in res:
                print('!!!! WARNING: Library request error !!!!')
                print(res)
                return
            if 'mediaItems' in res:
                numPics += len(res['mediaItems'])
    f.close()
    return numPics
    
def get_all_category_pics():
    print('Downloading pics with category filter...')
    for category in CATEGORIES:
        get_category_pics(category)
    print('Download complete!')

In [4]:
testID='AMyo5r0lZANxOJrBM7XH887PZfWTyK_x6LgX_n51XULlarUSqYiLm8g-L4xEEsX08zcy2HKmEDI8iMp09XTA0RCHYCR0MfxB7w'
res = requestIMG(testID)
downloadIMG(res['baseUrl'], 'testIMG.jpg')
 

NameError: name 'requestIMG' is not defined

## CompreFace Request

To request the CompreFace api, we need to first request the media item from the Google Photos API, which validates the baseUrl for 60 minutes, then download the image and then send this file in our request.

In [5]:
"""
    Request image with imgID
    We can only download the image from baseURL if we have requested
    the url from GP API and make our download request within 60 minutes
"""

def requestIMG(imgID):
    url = 'https://photoslibrary.googleapis.com/v1/mediaItems/'+imgID
    headers = {
        'content-type': 'application/json',
        'Authorization': 'Bearer {}'.format(CREDS.token)
    }
    res = requests.request("GET", url, headers=headers)
    return res.json()

"""
    Download image and place in WD
"""
def downloadIMG(url, file_name='imgToRecognize.jpg'):
    downloadResponse = requests.get(url)
    destination_folder = './downloads/'
    with open(os.path.join(destination_folder, file_name), 'wb') as f:
        f.write(downloadResponse.content)
        f.close()

"""
    Request the CompreFace API to recognize faces
"""
def recognizeFace(url):
    downloadIMG(url)
    headers = {
        'x-api-key': '0bedc62b-b2a4-4eb2-8efd-b62cc275e23c',
    }

    files = {
        'file': open('./downloads/imgToRecognize.jpg', 'rb'),
    }

    res = requests.post('http://localhost:8000/api/v1/recognition/recognize?face_plugins=landmarks, gender, age', headers=headers, files=files)
    return res.json()


## Build matrices

Build the takerSubject, picturedWith, and month-based matrices. The values of each cell is a string of comma separated item IDs which we'll hopefully use to request pictures in our final visualizations

In [6]:
import pandas as pd
import numpy as np
import random


RECOGNITION_THRESHOLD = .8 ## The similarity above which we allow a recognition

def getPictureTaker(imgID):
    GPRes = requestIMG(imgID) 
    try:
        mediaMetadata = GPRes['mediaMetadata']
    except:
        print('!!!! WARNING: GPRes error while trying to get picture taker')
        print(GPRes)
        errorCode = GPRes['error']['code']
        if errorCode == 401:
            init_gp_server()
            GPRes = requestIMG(imgID)
            mediaMetadata = GPRes['mediaMetadata']
        elif errorCode == 429:
            print('GP quota reached. Writing matrices to file...')
            return '429', ''
            
        else:
            return 'video', ''
    if 'photo' in mediaMetadata:
        photo = mediaMetadata['photo']
        if 'cameraModel' in photo:
            phoneType = photo['cameraModel']
            try:
                return PHONES_TO_PERSON[phoneType], GPRes['baseUrl']
            except:
                print('!!!! WARNING: Unrecognized camera')
                print(GPRes)
                downloadIMG(GPRes['baseUrl'], 'pictureTakerErr.jpg')
                return 'video', ''
        else:
            return 'jiusus', GPRes['baseUrl']
    else:
        return 'video', ''

"""
    Identify the faces in an image. Given the picture taker, increment the edge between
    pictureTaker and the face in the image.
    
"""
def processRecognition(res, pictureTaker, matrices, imgID, month=None):
    picturedWithMatrix, takerSubjectMatrix, subjectCounts, pictureOfSubjectByMonth = matrices
    if 'result' not in res:
        return
    results = res['result']
    subjects = []
    for result in results: ## Iterates through every face in picture
        possibleSubjects = result['subjects']
        if len(possibleSubjects) == 0:
            continue
        else:
            if possibleSubjects[0]['similarity'] < RECOGNITION_THRESHOLD:
                continue
            else:
                photoSubject = possibleSubjects[0]['subject']
                if photoSubject not in NAMES:
                    photoSubject = 'other'
                if month is None:
                    subjects.append(photoSubject)
                    takerSubjectMatrix.at[pictureTaker, photoSubject] += imgID+','
                    subjectCounts[photoSubject]['asSubject'] += 1
                    subjectCounts[pictureTaker]['asPhototaker'] += 1
                else:
                    pictureOfSubjectByMonth.at[photoSubject, month] += imgID+','
    subject_i, subject_j = 0, 1
    if month is None:
        while subject_i < len(subjects):
            firstSubject = subjects[subject_i]
            while subject_j < len(subjects):
                secondSubject = subjects[subject_j]
                picturedWithMatrix.at[firstSubject, secondSubject] += imgID+','
                picturedWithMatrix.at[secondSubject, firstSubject] += imgID+',' ## Make matrix symmetric for convenience
                subject_j += 1
            subject_i += 1
            subject_j = subject_i + 1

def createSubjectMatrices():
    picturedWithMatrix, takerSubjectMatrix = getMatrix('picturedWith'), getMatrix('takerSubject')
    overallStats = getOverallStatsJSON()
    print('Building subject-taker and photographed with matrices...')
    idFile = open('idFiles/picIDs.txt', 'r')
    start = int(getStart("subject"))
    ids = idFile.readlines()[start:]
    
    for entry in enumerate(ids):
        imgIDNum, imgID = entry[0]+start, entry[1][:-1]
        (pictureTaker, url) = getPictureTaker(imgID) # Cut out the EOL token
        if pictureTaker == '429':
            writeMatrix(picturedWithMatrix, 'picturedWith')
            writeMatrix(takerSubjectMatrix, 'takerSubject')
            writeStart(imgIDNum, 'subject')
            return
        if pictureTaker == 'video':
            continue
        else:
            try:
                recognitionRes = recognizeFace(url)
                matrices = (picturedWithMatrix, takerSubjectMatrix, overallStats, None)
                processRecognition(recognitionRes, pictureTaker, matrices, imgID)
            except Exception as e:
                print("!!!! WARNING: recognition api call failure in subject matrices creation !!!!")
                print(e)
                print(recognitionRes)
        imgIDNum += 1
    idFile.close()
    writeMatrix(picturedWithMatrix, 'takerSubject')
    writeMatrix(takerSubjectMatrix, 'picturedWith')
    writeStart(0, 'subject')
    print('Matrices built!')

def randomizeIDs(idString):
    idList = idString.split(",")
    random.shuffle(idList)
    idList = [i for i in idList if i]
    return ','.join(idList)
    
def createMonthMatrices():
    pictureBySubjectByMonth = getMatrix('pictureBySubjectByMonth')
    pictureOfSubjectByMonth = getMatrix('pictureOfSubjectByMonth')
    print('Building monthly matrices...')
    start = getStart('months')
    monthStart, linveStart = [int(i) for i in start.split(",")]
    for monthNum, monthEntry in enumerate(MONTHS[monthStart:]):
        mvvgIDNum, imgID = imgIDNum + lineStart, imgID[:-1] # Cut out the EOL token
        (pictureTaker, url) = getPictureTaker(imgID)
        if pictureTaker == 'video':
             continue
        elif pictureTaker == '429':
            writeMatrix(pictureBySubjectByMonth, 'pictureBySubjectByMonth')
            writeMatrix(pictureOfSubjectByMonth, 'pictureOfSubjectByMonth')
            writeStart(str(monthNum)+','+str(imgIDNum), 'months')
            return
        pictureBySubjectByMonth.at[pictureTaker, monthEntry['name']]+=imgID+','
        try:
            recognitionRes = recognizeFace(url)
            matrices = (None, None, None, pictureOfSubjectByMonth)
            processRecognition(recognitionRes, pictureTaker, matrices, imgID, monthEntry['name'])
        except Exception as e:
            print("!!!! WARNING: recognition api call failure in month matrix creation !!!!")
            print(e)
            print(recognitionRes)
    writeMatrix(pictureBySubjectByMonth, 'pictureBySubjectByMonth')
    writeMatrix(pictureOfSubjectByMonth, 'pictureOfSubjectByMonth')
    writeStart(str(0)+','+str(0), 'months')
    print('Matrices built!')
        
def createCategoryMatrix():
    subjectCategory = getMatrix('subjectCategory')
    print('Building category matrix...')
    start = getStart('categories')
    categoryStart, lineStart = [int(i) for i in start.split(",")]
    for categoryNum, category in enumerate(CATEGORIES[categoryStart:]):
        idFile = open('idFiles/categories/'+category[-1]+'PicIDs.txt', 'r')
        categoryNum += categoryStart
        ids = idFile.readlines()[lineStart:]
        for imgIDNum, imgID in enumerate(ids):
            imgIDNum, imgID = imgIDNum + lineStart, imgID[:-1]
            (pictureTaker, _) = getPictureTaker(imgID)
            if pictureTaker == 'video':
                 continue
            elif pictureTaker == '429':
                writeMatrix(subjectCategory, 'subjectCategory')
                writeStart(str(categoryNum)+','+str(imgIDNum), 'categories')
                return
            subjectCategory.at[pictureTaker, category[-1]]+=imgID+','
        idFile.close()
    writeMatrix(subjectCategory, 'subjectCategory')
    writeStart(str(0)+','+str(0), 'categories')
    print('Matrix built!')
    
"""
    Write the given matrices to csvs with the given names, and write the next run's starting
    index to the file with the given startingIndexName.
"""
def writeMatrix(matrix, name):
    matrix = matrix.applymap(randomizeIDs)
    if name == 'takerSubject':
        matrix.T.to_csv("data/subjectTaker.csv")
    matrix.to_csv("data/%s.csv" %name)

def writeStart(start, startingIndexName):
    with open('idFiles/startingIndices/%s.txt' %startingIndexName, 'w+') as f:
        f.write('%s' %start)
    f.close()

"""
    Reset the given matrices by writing empty matrices to the csvs with the given names.
"""

def resetMatrix(rowLabels, columnLabels, indexName, fileName):
    matrix = pd.DataFrame('', index=rowLabels, columns=columnLabels)
    matrix.index.name = indexName
    matrix.to_csv('data/%s.csv' %fileName)

def resetJSON():
    writeOverallStatsJSON(emptyOverallStats)

def getMatrix(fileName):
    
    matrix = pd.read_csv('data/%s.csv' %fileName, index_col=0)
    matrix = matrix.fillna('')
    return matrix
              
def getOverallStatsJSON():
    overallStats = {}
    with open('data/overallStats.json') as f_in:
        overallStats = json.load(f_in)
    f_in.close()
    return overallStats

def writeOverallStatsJSON(overallStats):
    with open("data/overallStats.json", "w") as fp:
        json.dump(overallStats , fp) 
    fp.close()
                             
def getStart(name):
    start = ''
    with open('idFiles/startingIndices/%s.txt' %name) as f:
        start = f.readlines()[0] 
    f.close()
    return start
    

Run all our cells to populate the matrices for the frontend

## Initialize overall stats

In [7]:
init_gp_server()





## Populate the subject matrices

In [8]:
takerSubject = getMatrix('takerSubject')
subjectTaker = takerSubject.T
subjectTaker.index.name = 'client'
writeMatrix(subjectTaker, 'subjectTaker')

## Populate the month defined matrices

In [52]:
init_gp_server()
resetMatrix(NAMES, [month['name'] for month in MONTHS], 'client', 'pictureBySubjectByMonth')
resetMatrix(NAMES, [month['name'] for month in MONTHS], 'client', 'picturedOfSubjectByMonth')

get_all_month_pics()
createMonthMatrices()

{'error': {'code': 401, 'message': 'Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.', 'status': 'UNAUTHENTICATED'}}
{'error': {'code': 401, 'message': 'Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.', 'status': 'UNAUTHENTICATED'}}
{'error': {'code': 401, 'message': 'Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.', 'status': 'UNAUTHENTICATED'}}
{'error': {'code': 401, 'message': 'Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential.

FileNotFoundError: [Errno 2] File b'data/pictureBySubjectByMonth' does not exist: b'data/pictureBySubjectByMonth'

## Populate the category matrix

In [50]:
init_gp_server()
#get_all_category_pics()
resetMatrix(NAMES, [category[-1] for category in CATEGORIES], 'client', 'subjectCategory')
createCategoryMatrix()


Building category matrix...
!!!! WARNING: Unrecognized camera
{'id': 'AMyo5r1sZxP4fk2iogLBvUdJUkBIUPnXTT2dft9DrutWYq8fN0uoKCGYOmSdXBPrl0FTbHYhZ8_faoM9iN6NSNy2__olL8n3AQ', 'productUrl': 'https://photos.google.com/lr/photo/AMyo5r1sZxP4fk2iogLBvUdJUkBIUPnXTT2dft9DrutWYq8fN0uoKCGYOmSdXBPrl0FTbHYhZ8_faoM9iN6NSNy2__olL8n3AQ', 'baseUrl': 'https://lh3.googleusercontent.com/lr/AFz2ejQbPeCsm6aUoW3alXmkN4UbIrM0w2JiAtpd_m_7C3YRiXJZWje6i8jZlBiq9yOBMg50dqBEgI8n4pTELr4h1KG1z5FaWlJz2QJDqpzXGguYPJv-4sJ4D3RKkklVM-zQJrhFxtdCskCsb2RV1e2MCUaaWOwb9ComdCH3SNWHczyX7RTMn1EioUKIgYfMWsh254TiwJbYAHyeRWOROaf9J7kKth354oT9sZgdL3ktzdj0QkquPKoeoMiabkQpBh9BWc6zpesAhY8Zv0bxKXgNsRB-C4eN2Am9R1DdLI5h8QF4WHlB1g6AgrfoImX1lpZNJHKRqljGWpC0iglgiLwwjUIYP2cQw3qdt2PpCc7ElZOzPPYORrrCpBERnLcPS_BycC7aSWE2BmDRSL7GLOizfUaEGpCZ5NdQ3ZCCAekW4lo3oWcAIw_AziX2kDIBOLC36ixajeiS-lpwff7Pwzhdn-jJrs8rRk3OBmOBt5R1xeOEN5KE23N517oTUfpXB7DyVtUF_EXI5vv_AECnLDs-up3damiA5YxlraJxAqydhAy-oy4Rd4NTjmCKa1kGcjMOheRRLJahTY8Cel_6F0VuEBecPUmLgudq1zHEg58v3BsC7ggUg

## Overall stats

In [ ]:
init_gp_server()
### Total pictures 
try:
    res = requests.request("GET", GET_LIBRARIES_URL, headers=PHOTOS_HEADERS)
    res.json()
    albumID = res.json()['albums'][1]['id']
except:
    print('!!!! WARNING: Library request error !!!!') 
    print(res)

res = requests.request("GET", GET_LIBRARIES_URL+'/'+albumID, headers=PHOTOS_HEADERS)
print(res)
res = res.json()
totalPictures = int(res['mediaItemsCount'])
overallCounts = getOverallCountsJSON()
overallCounts['total'] = totalPictures
writeOverallStatsJSON(overallCounts)
